In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('client_data.csv')

In [3]:
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [4]:
df.head(5)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,True,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,NaN,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,False,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,False,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,False,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,NaN,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,False,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0


In [5]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.000000,0.000000,44.266931,0.000000,0.000000
1,6c9103b3a9b68b340b12a03b7f027df9,2015-01-01,0.129444,0.106863,0.075004,40.565969,24.339581,16.226389
2,74ee25a056aa668b3e8eee92673c7ba8,2015-01-01,0.127566,0.105428,0.073487,40.565969,24.339581,16.226389
3,999c6f03d0d1bded3e39fa468276410b,2015-01-01,0.124942,0.102818,0.069076,40.565969,24.339581,16.226389
4,0ccc2948934ae51df8fb79d2f7ea4304,2015-01-01,0.127053,0.104382,0.072394,40.728885,24.437330,16.291555


In [6]:
price_df.isnull().sum()

id                    0
price_date            0
price_off_peak_var    0
price_peak_var        0
price_mid_peak_var    0
price_off_peak_fix    0
price_peak_fix        0
price_mid_peak_fix    0
dtype: int64

### Sum of price for energy and power along with the difference with respect to the period
The changes in the price from one period to another is a good indicator for churn. The expected thing to notice is that SME who churned will have a positive difference meaning that the prices for energy and power are getting high. The inverse is expected for SME who did not churn.

In [7]:
price_df['price_p1']=price_df['price_off_peak_var']+price_df['price_off_peak_fix']
price_df['price_p2']=price_df['price_peak_var']+price_df['price_peak_fix']
price_df['price_p3']=price_df['price_mid_peak_var']+price_df['price_mid_peak_fix']
price_df['pp12']=price_df['price_p2']-price_df['price_p1']
price_df['pp23']=price_df['price_p3']-price_df['price_p2']
price_df['pp13']=price_df['price_p3']-price_df['price_p1']

In [8]:
price_df.isnull().sum()

id                    0
price_date            0
price_off_peak_var    0
price_peak_var        0
price_mid_peak_var    0
price_off_peak_fix    0
price_peak_fix        0
price_mid_peak_fix    0
price_p1              0
price_p2              0
price_p3              0
pp12                  0
pp23                  0
pp13                  0
dtype: int64

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [9]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff_1 = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.rename(columns={'price_off_peak_var': 'jan_1', 'price_off_peak_fix': 'jan_2'}).drop(columns='price_date'), on='id')
diff_1['diff_dec_january_energy_p1'] = diff_1['dec_1'] - diff_1['jan_1']
diff_1['diff_dec_january_power_p1'] = diff_1['dec_2'] - diff_1['jan_2']
diff_1 = diff_1[['id', 'diff_dec_january_energy_p1','diff_dec_january_power_p1']]
diff_1.head()

,id,diff_dec_january_energy_p1,diff_dec_january_power_p1
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [11]:
# Difference between peak prices in December and Preceding January
# Group peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_peak_var': 'mean', 'price_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff_2 = pd.merge(dec_prices.rename(columns={'price_peak_var': 'dec_1', 'price_peak_fix': 'dec_2'}), jan_prices.rename(columns={'price_peak_var': 'jan_1', 'price_peak_fix': 'jan_2'}).drop(columns='price_date'), on='id')
diff_2['diff_dec_january_energy_p2'] = diff_2['dec_1'] - diff_2['jan_1']
diff_2['diff_dec_january_power_p2'] = diff_2['dec_2'] - diff_2['jan_2']
diff_2 = diff_2[['id', 'diff_dec_january_energy_p2','diff_dec_january_power_p2']]
diff_2.head()

,id,diff_dec_january_energy_p2,diff_dec_january_power_p2
0,0002203ffbb812588b632b9e628cc38d,-0.002302,0.097749
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,-0.005120,0.097749
4,00114d74e963e47177db89bc70108537,0.000000,0.000000


In [12]:
# Difference between mid peak prices in December and preceding January
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_mid_peak_var': 'mean', 'price_mid_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff_3 = pd.merge(dec_prices.rename(columns={'price_mid_peak_var': 'dec_1', 'price_mid_peak_fix': 'dec_2'}), jan_prices.rename(columns={'price_mid_peak_var': 'jan_1', 'price_mid_peak_fix': 'jan_2'}).drop(columns='price_date'), on='id')
diff_3['diff_dec_january_energy_p3'] = diff_3['dec_1'] - diff_3['jan_1']
diff_3['diff_dec_january_power_p3'] = diff_3['dec_2'] - diff_3['jan_2']
diff_3 = diff_3[['id', 'diff_dec_january_energy_p3','diff_dec_january_power_p3']]
diff_3.head()

,id,diff_dec_january_energy_p3,diff_dec_january_power_p3
0,0002203ffbb812588b632b9e628cc38d,0.003487,0.065166
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,0.000763,0.065166
4,00114d74e963e47177db89bc70108537,0.000000,0.000000
